# Objectives
1. Observe progression of covid-19 by continent and country
2. Investigate whether current level of vaccination is slowing down the spread or not
3. Determine where the vaccine is needed the most by looking at mortality rate

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
from matplotlib import pyplot as plt
from collections import Counter, OrderedDict
import pycountry_convert as pc
from collections import OrderedDict

In [2]:
covid = pd.read_csv(r"C:\Users\taewoo\PycharmProjects\data_professionals\Projects\Datasets\Covid-19 Activity.csv")
covid.info()

c:\users\taewoo\pycharmprojects\data-analysis-practice\venv\interpreter\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1687420 entries, 0 to 1687419
Data columns (total 13 columns):
 #   Column                           Non-Null Count    Dtype  
---  ------                           --------------    -----  
 0   PEOPLE_POSITIVE_CASES_COUNT      1687420 non-null  int64  
 1   COUNTY_NAME                      1558424 non-null  object 
 2   PROVINCE_STATE_NAME              1585556 non-null  object 
 3   REPORT_DATE                      1687420 non-null  object 
 4   CONTINENT_NAME                   1685516 non-null  object 
 5   DATA_SOURCE_NAME                 1687420 non-null  object 
 6   PEOPLE_DEATH_NEW_COUNT           1687420 non-null  int64  
 7   COUNTY_FIPS_NUMBER               1531768 non-null  float64
 8   COUNTRY_ALPHA_3_CODE             1685516 non-null  object 
 9   COUNTRY_SHORT_NAME               1687420 non-null  object 
 10  COUNTRY_ALPHA_2_CODE             1685040 non-null  object 
 11  PEOPLE_POSITIVE_NEW_CASES_COUNT  1687420 non-null 

In [3]:
covid.head(10)

,PEOPLE_POSITIVE_CASES_COUNT,COUNTY_NAME,PROVINCE_STATE_NAME,REPORT_DATE,CONTINENT_NAME,DATA_SOURCE_NAME,PEOPLE_DEATH_NEW_COUNT,COUNTY_FIPS_NUMBER,COUNTRY_ALPHA_3_CODE,COUNTRY_SHORT_NAME,COUNTRY_ALPHA_2_CODE,PEOPLE_POSITIVE_NEW_CASES_COUNT,PEOPLE_DEATH_COUNT
0,18046,Kenosha,Wisconsin,2021-04-26,America,New York Times,0,55059.0,USA,United States,US,3,321
1,18096,Kenosha,Wisconsin,2021-04-27,America,New York Times,0,55059.0,USA,United States,US,50,321
2,18123,Kenosha,Wisconsin,2021-04-28,America,New York Times,1,55059.0,USA,United States,US,27,322
3,18148,Kenosha,Wisconsin,2021-04-29,America,New York Times,0,55059.0,USA,United States,US,25,322
4,18163,Kenosha,Wisconsin,2021-04-30,America,New York Times,0,55059.0,USA,United States,US,15,322
5,18188,Kenosha,Wisconsin,2021-05-01,America,New York Times,0,55059.0,USA,United States,US,25,322
6,18212,Kenosha,Wisconsin,2021-05-02,America,New York Times,0,55059.0,USA,United States,US,24,322
7,18212,Kenosha,Wisconsin,2021-05-03,America,New York Times,0,55059.0,USA,United States,US,0,322
8,18239,Kenosha,Wisconsin,2021-05-04,America,New York Times,1,55059.0,USA,United States,US,27,323
9,18251,Kenosha,Wisconsin,2021-05-05,America,New York Times,0,55059.0,USA,United States,US,12,323


### Data Cleansing

In [4]:
# dropping columns that aren't too useful for this analysis
covid.drop(columns=['COUNTY_FIPS_NUMBER', 'DATA_SOURCE_NAME', 'COUNTRY_ALPHA_2_CODE', 'COUNTRY_ALPHA_3_CODE'], inplace=True)
covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1687420 entries, 0 to 1687419
Data columns (total 9 columns):
 #   Column                           Non-Null Count    Dtype 
---  ------                           --------------    ----- 
 0   PEOPLE_POSITIVE_CASES_COUNT      1687420 non-null  int64 
 1   COUNTY_NAME                      1558424 non-null  object
 2   PROVINCE_STATE_NAME              1585556 non-null  object
 3   REPORT_DATE                      1687420 non-null  object
 4   CONTINENT_NAME                   1685516 non-null  object
 5   PEOPLE_DEATH_NEW_COUNT           1687420 non-null  int64 
 6   COUNTRY_SHORT_NAME               1687420 non-null  object
 7   PEOPLE_POSITIVE_NEW_CASES_COUNT  1687420 non-null  int64 
 8   PEOPLE_DEATH_COUNT               1687420 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 115.9+ MB


In [5]:
covid.isnull().sum()

PEOPLE_POSITIVE_CASES_COUNT             0
COUNTY_NAME                        128996
PROVINCE_STATE_NAME                101864
REPORT_DATE                             0
CONTINENT_NAME                       1904
PEOPLE_DEATH_NEW_COUNT                  0
COUNTRY_SHORT_NAME                      0
PEOPLE_POSITIVE_NEW_CASES_COUNT         0
PEOPLE_DEATH_COUNT                      0
dtype: int64

In [6]:
def country_to_continent(country_name):
    '''
    return continent of a country
    
        Param:
            country_name (str) = name of a country
        Return:
            continent_name (str) = name of a continent
    '''
    country_alpha2 = pc.country_name_to_country_alpha2(country_name)
    country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
    continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
    return continent_name

# drop duplicates
covid.drop_duplicates(inplace=True)
# extract countries with nontype continent
country_of_nan_continent = covid[covid['CONTINENT_NAME'].isnull()]['COUNTRY_SHORT_NAME'].unique() 
# fill continents
for country in country_of_nan_continent:
    country_index = covid.loc[covid['COUNTRY_SHORT_NAME'] == country].index
    covid.loc[country_index, 'CONTINENT_NAME'] = country_to_continent(country)

# it's possible to find province if county can be found, not possible vice versa
print('county without null, province with null')
print(covid.loc[covid['COUNTY_NAME'].notnull() & covid['PROVINCE_STATE_NAME'].isnull()].shape[0])
print('county with null, province with null')
print(covid.loc[covid['COUNTY_NAME'].isnull() & covid['PROVINCE_STATE_NAME'].isnull()].shape[0])
print('county with null, province without null')
print(covid.loc[covid['COUNTY_NAME'].isnull() & covid['PROVINCE_STATE_NAME'].notnull()].shape[0])
print()

# I was unable to identify provinces and counties
covid.fillna('Unknown', inplace=True)
print(covid.isnull().sum())

county without null, province with null
0
county with null, province with null
101864
county with null, province without null
27132

PEOPLE_POSITIVE_CASES_COUNT        0
COUNTY_NAME                        0
PROVINCE_STATE_NAME                0
REPORT_DATE                        0
CONTINENT_NAME                     0
PEOPLE_DEATH_NEW_COUNT             0
COUNTRY_SHORT_NAME                 0
PEOPLE_POSITIVE_NEW_CASES_COUNT    0
PEOPLE_DEATH_COUNT                 0
dtype: int64


## Total cases of Covid-19 by countries and provinces

In [7]:
def provincial_case(df, country):
    '''
    return a dictionary of all provincial cases
    
        Args:
            df (DataFrame): a dataframe with all information of countries and provinces
            country (str): any country in the dataframe
        Return:
            (OrderedDict): provinces and total cases
    '''
    country_table = df.loc[df['COUNTRY_SHORT_NAME'] == country]
    province_list = country_table['PROVINCE_STATE_NAME'].unique()
    province_dict = dict()
    for province in province_list:
        province_dict[province] = country_table.loc[country_table['PROVINCE_STATE_NAME'] == province]['PEOPLE_POSITIVE_CASES_COUNT'].max()
    return OrderedDict(sorted(province_dict.items(), key=lambda x: x[1], reverse=True))


def country_case(df, country):
    '''
    return the sum of *confirmed* cases of Covid-19 in a country
    
        Args:
            df (DataFrame): a dataframe with all information of countries
            country (str): any country in the dataframe
        Return:
            country (str): input country
            total (int): total of confirmed cases 
    '''
    country_df = df.loc[df['COUNTRY_SHORT_NAME'] == country]
    total = country_df['PEOPLE_POSITIVE_NEW_CASES_COUNT'].sum()
    return country, total


every_country = covid['COUNTRY_SHORT_NAME'].unique()
country_cases = dict()

for country in every_country:
    country, total_case = country_case(covid, country)
    country_cases[country] = total_case

country_cases = OrderedDict(sorted(country_cases.items(), key=lambda x: x[1], reverse=True))

print('Total cases of Covid-19 by countries')
print(country_cases)
print()
print('Test - Provincial cases in Canada')
print(provincial_case(covid, 'Canada'))

Total cases of covid-19 by countries
OrderedDict([('United States', 32772514), ('India', 22992517), ('Brazil', 15209990), ('France', 5730585), ('Turkey', 5044936), ('Russia', 4832959), ('United Kingdom', 4437217), ('Italy', 4116287), ('Spain', 3581392), ('Germany', 3538208), ('Argentina', 3165121), ('Colombia', 3015301), ('Poland', 2835083), ('Iran', 2673219), ('Mexico', 2366496), ('Ukraine', 2178095), ('Peru', 1850290), ('Indonesia', 1718575), ('Czechia', 1645448), ('South Africa', 1597724), ('Netherlands', 1565880), ('Canada', 1302489), ('Chile', 1252808), ('Iraq', 1117627), ('Philippines', 1108826), ('Romania', 1066731), ('Belgium', 1017876), ('Sweden', 1007792), ('Pakistan', 864557), ('Portugal', 839740), ('Israel', 838957), ('Hungary', 792386), ('Bangladesh', 775027), ('Jordan', 720998), ('Serbia', 701326), ('Switzerland', 674296), ('Japan', 647003), ('Austria', 631896), ('United Arab Emirates', 537524), ('Lebanon', 533141), ('Morocco', 513922), ('Malaysia', 444484), ('Saudi Arabi

### 10 countries with the most positive cases of Covid-19

In [28]:
print('10 countries with the highest cases \n')
for i, item in enumerate(country_cases):
    if i < 10:
        print(f'Top {i+1} - {item}: {country_cases[item]}')
    else:
        break

10 countries with the highest cases 

Top 1 - United States: 32772514
Top 2 - India: 22992517
Top 3 - Brazil: 15209990
Top 4 - France: 5730585
Top 5 - Turkey: 5044936
Top 6 - Russia: 4832959
Top 7 - United Kingdom: 4437217
Top 8 - Italy: 4116287
Top 9 - Spain: 3581392
Top 10 - Germany: 3538208


Since the *U.S.* is the country with **the most cases**, let's take a look at the positive cases by provinces within the country.

In [30]:
print(provincial_case(covid, 'United States'))

OrderedDict([('California', 1236461), ('New York', 938087), ('Illinois', 543111), ('Arizona', 542072), ('Florida', 491027), ('Texas', 395522), ('Nevada', 246811), ('Michigan', 159802), ('Pennsylvania', 149953), ('Utah', 149164), ('Massachusetts', 133645), ('Ohio', 125964), ('Minnesota', 122024), ('Wisconsin', 115913), ('North Carolina', 111634), ('Washington', 105425), ('New Jersey', 103352), ('Georgia', 101569), ('Indiana', 100112), ('Connecticut', 99161), ('Missouri', 98627), ('Tennessee', 98606), ('Rhode Island', 96768), ('Oklahoma', 86408), ('Maryland', 83944), ('Kentucky', 81644), ('Alabama', 76544), ('Virginia', 76438), ('South Carolina', 74312), ('Colorado', 71672), ('Nebraska', 71259), ('Delaware', 62805), ('Kansas', 58303), ('New Mexico', 57582), ('Iowa', 57447), ('Idaho', 52025), ('District of Columbia', 48195), ('Louisiana', 46200), ('Arkansas', 39203), ('Oregon', 38089), ('New Hampshire', 35389), ('South Dakota', 31535), ('Alaska', 30259), ('Hawaii', 26115), ('North Dakota'

California, the state with the highest population has experienced extremely high number of cases

### Investigating the progression of COVID-19 from Jan - July 2020 by continent
**Goals**
* Group data by continents and time
* Use month as the time measurement unit
* Create a newly reported cases graph

In [ ]:
covid['YEAR'] = pd.to_datetime(covid['REPORT_DATE']).dt.year
covid['MONTH'] = pd.to_datetime(covid['REPORT_DATE']).dt.month
continent_new_case = covid[['CONTINENT_NAME', 'PEOPLE_POSITIVE_NEW_CASES_COUNT','YEAR', 'MONTH']].groupby(
                       ['CONTINENT_NAME','YEAR', 'MONTH'], as_index=False).sum()
print(continent_new_case)

To display all this information on the same graph, let's drop Asia's Dec of 2019 and add Africa's January of 2020.

Then, I can start making graphs

### Positive Cases of each Continent

In [ ]:
continent_new_case.drop(index=13, inplace=True)
continent_new_case = continent_new_case.append({'CONTINENT_NAME':'Africa', 'YEAR':2020, 'MONTH':1, 'PEOPLE_POSITIVE_NEW_CASES_COUNT':0}, ignore_index=True)
continent_new_case.sort_values(by=['CONTINENT_NAME', 'MONTH'], inplace=True)
continent_new_case.reset_index(drop=True, inplace=True)

In [ ]:
print(continent_new_case[['CONTINENT_NAME', 'MONTH', 'PEOPLE_POSITIVE_NEW_CASES_COUNT']])

In [ ]:
continent_list = continent_new_case['CONTINENT_NAME'].unique()
color_list = ['tab:purple', 'tab:red', 'tab:blue', 'tab:green', 'tab:orange']
plt.style.use('bmh')
plt.rcParams['figure.figsize'] = [18, 10]

# use for loop to create a line graph for each continent
for continent, color in zip(continent_list, color_list):
    plt.plot('MONTH', 'PEOPLE_POSITIVE_NEW_CASES_COUNT', data=continent_new_case.loc[continent_new_case['CONTINENT_NAME']==continent], c=color, label=continent, marker='o', linewidth=3)

# create a list of sum of cases for each continent in July (last month)
max_continent = []
for continent in continent_list:
    max_continent.append(continent_new_case.loc[(continent_new_case['CONTINENT_NAME'] == continent) & 
                                                (continent_new_case['MONTH'] == 7)]['PEOPLE_POSITIVE_NEW_CASES_COUNT'].sum())

# create and position labels
for y in max_continent:
    if y >= 1000000:
        plt.text(7.05, y, str(round(y/1000000, 2)) +'M')
    else:
        plt.text(7.05, y, str(round(y/1000)) +'K')
plt.legend()
plt.ylim(-100000, 3000000)
plt.xlim(1, 7.3)
plt.xlabel('Month')
plt.title('Newly Reported Cases of Covid-19 for each Continent')
plt.ylabel('Number of Infectee (M)')
plt.show()

To be clear each plot **doesn't** represent the sum of previous cases + new cases. It's a quantity of newly reported cases.

* Initially, Asia experienced the fastest growth rate as Wuhan is the place of origin of Covid-19
* The number in Europe spiked up in between February and April then experienced a consistent decline in growth rate
* America started slowly, then has been experiencing the fastest growth rate (it's on-going)
* Growth rates in both Africa and Oceania have been steadily and very slowly increasing

In [ ]:
# create a list of total of new cases for each time period
aggregate_cases = []
total = 0
for continent in continent_list:
    for item, new_case in zip(continent_new_case['CONTINENT_NAME'], continent_new_case['PEOPLE_POSITIVE_NEW_CASES_COUNT']):
        if continent == item:
            total += new_case
            aggregate_cases.append(total)
        else:
            total = 0

# add it as a new column
continent_new_case['AGGREGATE_CASES'] = aggregate_cases

In [ ]:
for continent, color in zip(continent_list, color_list):
    plt.plot('MONTH', 'AGGREGATE_CASES', data=continent_new_case.loc[continent_new_case['CONTINENT_NAME']==continent], c=color, label=continent, marker='o', linewidth=3)

max_continent = []
for continent in continent_list:
    max_continent.append(continent_new_case.loc[(continent_new_case['CONTINENT_NAME'] == continent) & 
                                                (continent_new_case['MONTH'] == 7)]['AGGREGATE_CASES'].sum())
for y in max_continent:
    if y >= 1000000:
        plt.text(7.05, y, str(round(y/1000000, 2)) +'M')
    else:
        plt.text(7.05, y, str(round(y/1000)) +'K')
plt.legend()
plt.xlabel('Month')
plt.xlim(1, 7.3)
plt.title('Sum of all cases of Covid-19 for each Continent')
plt.ylabel('Number of Infectee (M)')
plt.show()

* America is by far the *most* dangerous continent to travel in the current state - **fastest growth rate** since March
* Asia and Africa are experiencing a steady increase
* Europe and Oceania are managing to flatten the curves

### Counting Deaths of each Continent

In [ ]:
continent_death = covid[['CONTINENT_NAME', 'PEOPLE_DEATH_NEW_COUNT', 'YEAR', 'MONTH']].groupby(
                       ['CONTINENT_NAME','YEAR', 'MONTH'], as_index=False).sum()
continent_death = continent_death.append({'CONTINENT_NAME':'Africa', 'YEAR':2020, 'MONTH':1, 'PEOPLE_DEATH_NEW_COUNT':0}, ignore_index=True)
continent_death.drop(index=13, inplace=True)
continent_death.sort_values(by=['CONTINENT_NAME', 'MONTH'], inplace=True)
continent_death.reset_index(drop=True, inplace=True)

DEATH_AGGREGATE = []
death_count = 0
for continent in continent_list:
    for item, death in zip(continent_death['CONTINENT_NAME'], continent_death['PEOPLE_DEATH_NEW_COUNT']):
        if continent == item:
            death_count += death
            DEATH_AGGREGATE.append(death_count)
        else:
            death_count = 0
continent_death['DEATH_AGGREGATE'] = DEATH_AGGREGATE
print(continent_death)

In [ ]:
for continent, color in zip(continent_list, color_list):
    plt.plot('MONTH', 'DEATH_AGGREGATE', data=continent_death.loc[continent_death['CONTINENT_NAME']==continent], c=color, label=continent, marker='o', linewidth=3)

max_death = []
for continent in continent_list:
    max_death.append(continent_death.loc[(continent_death['CONTINENT_NAME'] == continent) & 
                                                (continent_death['MONTH'] == 7)]['DEATH_AGGREGATE'].sum())

for y in max_death:
    if y >= 1000:
        plt.text(7.05, y, str(round(y/1000)) +'K')
    else:
        plt.text(7.05, y, str(y))
plt.legend()
plt.xlim(1, 7.3)
plt.xlabel('Month')
plt.title('Death Toll for each Continent')
yticks = [50000, 100000, 150000, 200000, 250000, 300000, 350000]
ylabels = ['50K', '100K', '150K', '200K', '250K', '300K', '350K']
plt.yticks(ticks=yticks, labels=ylabels)
plt.ylabel('Fatality')
plt.show()

* As observed from previous graphs, the decline of Covid-19's positive cases in Europe has resulted in gentler slopes after April
* The extreme growth rate of positive cases in America is reflected by the steep slope on this graph
* Although the total of positive cases in Asia is greater than that of Europe by 80,000, death toll is much lower (it could be the consequence of how well the countries handled the situation)

### Mortality Rate

In [ ]:
def death_rate(df, continent):
    '''
    Args:
        df: DataFrame
        header: str or a list of str, a header of a column 
    Return:
         death rate of a continent or an average death rate of continents
    '''
    # if calculating death rate of a continent, find the lastest updates of death and infected population
    if type(continent) == str:
        temp = df.loc[df['CONTINENT_NAME'] == continent]
        return '{}%'.format(round(temp['PEOPLE_DEATH_COUNT'].max()/temp['PEOPLE_POSITIVE_CASES_COUNT'].max()*100, 2))
    # if calculating death rate of continents, find each continent's death rate and find the average
    else:
        death = []
        population = []
        total = 0
        for item in continent:
            death.append(df.loc[df['CONTINENT_NAME'] == item]['PEOPLE_DEATH_COUNT'].max())
            population.append(df.loc[df['CONTINENT_NAME'] == item]['PEOPLE_POSITIVE_CASES_COUNT'].max())
        for d, p in zip(death, population):
            total += d/p*100
        return '{}%'.format(round(total/len(death), 2))

print(death_rate(covid,'Africa'))
print(death_rate(covid,'America'))
print(death_rate(covid,['Africa', 'America']))

In [ ]:
for continent in continent_list:
    print(f"Mortality rate of Covid-19 in {continent}: {death_rate(covid, continent)}")
print(f'===============\nAverage mortality rate of Covid-19: {death_rate(covid, continent_list)}')

*Europe* has recorded the highest mortality rate of **5.77%** while *Oceania* has managed the lowest of **0.99%**.

Overall, the average is **2.88%**, which is considered **not fatal**.

In [ ]:
total_all = covid[['MONTH', 'PEOPLE_POSITIVE_NEW_CASES_COUNT', 'PEOPLE_DEATH_NEW_COUNT']].groupby(['MONTH']).sum()
total_all.drop(12, inplace=True)
total_positive_list = []
total_death_list = []
count_positive = 0
count_death = 0
for positive, death in zip(total_all['PEOPLE_POSITIVE_NEW_CASES_COUNT'], total_all['PEOPLE_DEATH_NEW_COUNT']):
    count_positive += positive
    count_death += death
    total_positive_list.append(count_positive)
    total_death_list.append(count_death)
total_all['TOTAL_POSITIVE_CASE'] = total_positive_list
total_all['TOTAL_DEATH'] = total_death_list
print(total_all)

In [ ]:
total_all['TOTAL_POSITIVE_CASE'].plot(linewidth=3, marker='o')
plt.title('All Positive Cases of Covid-19')
#setting up y-axis and x-axis of all positive cases
plt.xlim(1, 7.3)
plt.ylim(-100000, 16000000)
yticks = [0, 2000000, 4000000, 6000000, 8000000, 10000000, 12000000, 14000000, 16000000]
ylabels = ['0', '2M', '4M', '6M', '8M', '10M', '12M', '14M', '16M']
plt.yticks(ticks=yticks, labels=ylabels)
y = total_all['TOTAL_POSITIVE_CASE'].max()
plt.text(6.85, y+200000, str(y))
plt.show()

total_all['TOTAL_DEATH'].plot(linewidth=3, marker='o')
plt.title('International Death Toll')
#setting up y-axis and x-axis of death toll
plt.xlim(1, 7.3)
plt.ylim(-10000, 700000)
yticks = [0, 100000, 200000, 300000, 400000, 500000, 600000, 700000]
ylabels = ['0', '100K', '200K', '300K', '400K', '500K', '600K', '700k']
plt.yticks(ticks=yticks, labels=ylabels)
y = total_all['TOTAL_DEATH'].max()
plt.text(6.9, y+10000, str(y))
plt.show()

In [ ]:
covid['REPORT_DATE'] = pd.to_datetime(covid['REPORT_DATE'])
covid.sort_values(by='REPORT_DATE', inplace=True)

### Plotting every point

In [ ]:
covid_daily = covid.groupby('REPORT_DATE', as_index=False).sum()
plt.plot_date(x=covid_daily['REPORT_DATE'], y=covid_daily['PEOPLE_POSITIVE_CASES_COUNT'])
yticks = [0, 2000000, 4000000, 6000000, 8000000, 10000000, 12000000, 14000000, 16000000]
ylabels = ['0', '2M', '4M', '6M', '8M', '10M', '12M', '14M', '16M']
plt.yticks(ticks=yticks, labels=ylabels)
plt.title('The Tracks of Covid-19')
plt.show()

Why is the last point out of place?

Since this graph is reporting the aggregate of all reported cases in the past, the last point cannot be where it is plotted.

Let's investigate

### Tracking down the problem

In [ ]:
print(covid_daily.tail(2))

In [ ]:
print(covid.loc[covid['REPORT_DATE']=='2020-07-23']['PEOPLE_POSITIVE_CASES_COUNT'].isnull().sum())
print(covid.loc[covid['REPORT_DATE'].isin(['2020-07-23', '2020-07-22'])]
              [['PEOPLE_POSITIVE_CASES_COUNT', 'CONTINENT_NAME', 'REPORT_DATE']]
              .groupby(['CONTINENT_NAME', 'REPORT_DATE']).sum())

In America, cases went from 8026207 to 4037927. Given it happened within a day, the number should've been more like 8037927

Let's see where in America it happened

In [ ]:
pd.set_option('display.max_rows', None)
print(covid.loc[(covid['REPORT_DATE'].isin(['2020-07-23', '2020-07-22'])) & (covid['CONTINENT_NAME'] == 'America')]
[['PEOPLE_POSITIVE_CASES_COUNT', 'COUNTRY_SHORT_NAME', 'REPORT_DATE']].groupby(['COUNTRY_SHORT_NAME', 'REPORT_DATE']).sum())

Everywhere looks fine except the U.S.

It was due to missing 2020-07-23 reports in the U.S.

Since the total on 2020-07-23 is highly affected by this factor, I'm going to graph til 2020-07-22

In [ ]:
# drop the reports on 2020-07-23
covid_temp = covid.copy()
covid_temp.drop(covid_temp.loc[covid_temp['REPORT_DATE'] == '2020-07-23'].index, inplace=True)
covid_daily = covid_temp.groupby('REPORT_DATE', as_index=False).sum()
plt.plot_date(x=covid_daily['REPORT_DATE'], y=covid_daily['PEOPLE_POSITIVE_CASES_COUNT'])
yticks = [0, 2000000, 4000000, 6000000, 8000000, 10000000, 12000000, 14000000, 16000000]
ylabels = ['0', '2M', '4M', '6M', '8M', '10M', '12M', '14M', '16M']
plt.yticks(ticks=yticks, labels=ylabels)
plt.title('The Tracks of Covid-19')
plt.show()

Now it looks great with no error

In [ ]:
covid_daily = covid.groupby('REPORT_DATE', as_index=False).sum()
covid_daily.drop(covid_temp.loc[covid_temp['REPORT_DATE'] == '2020-07-23'].index, inplace=True)
plt.plot_date(x=covid_daily['REPORT_DATE'], y=covid_daily['PEOPLE_POSITIVE_CASES_COUNT'])
yticks = [0, 2000000, 4000000, 6000000, 8000000, 10000000, 12000000, 14000000, 16000000]
ylabels = ['0', '2M', '4M', '6M', '8M', '10M', '12M', '14M', '16M']
plt.yticks(ticks=yticks, labels=ylabels)
plt.title('The Tracks of Covid-19')
plt.show()